In [5]:
import json
import cobrakbase

In [24]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
kbase = cobrakbase.KBaseAPI(token)
kbase_dev = cobrakbase.KBaseAPI(token, dev=True)

In [3]:
model1 = kbase.get_object('iML1515.kb', 'filipeliu:narrative_1504192868437')

In [4]:
model2 = kbase.get_object('GCF_000005845.2.mdl', 'filipeliu:narrative_1504192868437')

In [7]:
with open('../data/html/data/model1.json', 'w') as fh:
    fh.write(json.dumps(model1))
with open('../data/html/data/model2.json', 'w') as fh:
    fh.write(json.dumps(model2))

In [8]:
ws_dev = 'filipeliu:narrative_1580723870549'

In [10]:
import escher
import modelseed_escher

In [11]:
escher_seed = modelseed_escher.EscherManager(escher)

In [13]:
escher_seed.list_datasets()

{'BIOS',
 'Bacillus subtilis',
 'Escherichia coli',
 'Homo sapiens',
 'KEGG',
 'ModelSEED',
 'Neurospora crassa',
 'Oscarichia coli',
 'Pichia pastoris',
 'Pseudomonas putida KT2440',
 'Saccharomyces cerevisiae',
 'Synechococcus',
 'consensus',
 'raw'}

In [62]:
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    map_data = escher_seed.get_map('ModelSEED', model_id, map_id)
    map_data.escher_data['map_name'] = map_id
    if not 'authors' in map_data.escher_data:
        map_data.escher_data['authors'] = []
    for rxn_uid in map_data.escher_graph['reactions']:
        rxn_node = map_data.escher_graph['reactions'][rxn_uid]
        rxn_node['reversibility'] = 1 if rxn_node['reversibility'] else 0
    
        #TEMP HACK FIX OBJECT SPEC
        for seg_uid in rxn_node['segments']:
            seg = rxn_node['segments'][seg_uid]
            if seg['b1'] == None:
                seg['b1'] = {'x' : 0, 'y' : 0}
            if seg['b2'] == None:
                seg['b2'] = {'x' : 0, 'y' : 0}
    for node_uid in map_data.escher_graph['nodes']:
        node = map_data.escher_graph['nodes'][node_uid]
        if 'node_is_primary' in node:
            node['node_is_primary'] = 1 if node['node_is_primary'] else 0
        if not 'node_is_primary' in node:
            node['node_is_primary'] = 0
        #TEMP HACK FIX OBJECT SPEC
        if not 'bigg_id' in node:
            node['bigg_id'] = ''
        if not 'label_x' in node:
            node['label_x'] = 0
        if not 'label_y' in node:
            node['label_y'] = 0
        if not 'name' in node:
            node['name'] = ''
    kbase_escher = {
        "metadata" : map_data.escher_data,
        "layout" : map_data.escher_graph
    }
    o_id = map_id.replace(',', ' ') \
                 .replace(')', ' ') \
                 .replace('(', ' ') \
                 .replace(' ', '_') \
                 .replace('__', '_')
    #print(map_data.escher_data['map_name'], o_id)
    kbase_dev.save_object(o_id, ws_dev, "KBaseFBA.EscherMap", kbase_escher)